<a href="https://colab.research.google.com/github/ingabLee/Transformers_book/blob/main/TransformerLearning_Chap56_65.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
dataset = [
    ["What music do you like?", "I like Rock music.", 1],
    ["What is your favorite food?", "I like sushi the best", 1],
    ["What is your favorite color?", "I'm going to be a doctor", 0],
    ["What is your favorite song?", "Tokyo olympic game in 2020 was postponed", 0],
    ["Do you like watching TV show?", "Yeah, I often watch it in my spare time", 1]
]

In [ ]:
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer
from torch.optim import adamw
from torch import nn

# define class
class BertEnsembleForNextSentencePrediction(BertPreTrainedModel):
  # creator
  def __init__(self, config, *args, **kwargs):
    super().__init__(config)

    #QA BERT Model
    self.bert_model_1 = BertModel(config)

    #AQ BERT Model
    self.bert_model_2 = BertModel(config)

    # linear function
    self.cls = nn.Linear(2*self.config.hidden_size, 2)

    # init weight
    self.init_weights()

  # forward
  def forward(
      self,
      input_ids=None,
      attention_mask=None,
      token_type_ids=None,
      position_ids=None,
      head_mask=None,
      inputs_embeds=None,
      next_sentence_label=None ):
    # outputs list
    outputs = []

    #save input_ids first text
    input_ids_1 = input_ids[0]

    # save input_ids attention_mask
    attention_mask_1 = attention_mask[0]

    #save outputs result that  bert_model_1 result
    outputs.append( self.bert_model_1(input_ids_1, attention_mask=attention_mask_1))

    #save second text(input_ids)
    input_ids_2 = input_ids[1]

    #save second text attenton_mask
    attention_mask_2 = attention_mask[1]

    #save result of bert_model_2 input_ids_2
    outputs.append(self.bert_model_2(input_ids_2, attention_mask=attention_mask_2))

    # outputs에 쌓인 output의 두번째 요소(output[1])를 하나씩 추출하여
    # torch.cat()으로 토치 텐서 형태로 병합
    # 이를 통해 마지막 은닉층 임베딩 상태를 구함.
    last_hidden_states = torch.cat([output[1] for output in outputs], dim=1)

    #self.cls 선형함수에 마지막 은닉층 임베딩 상태를 투입하여 로짓 추출.
    logits = self.cls(last_hidden_states)

    # crossentropy loss
    if next_sentence_label is not None:
      # nn.CrossEntropyLoss() 입력 데이타의 마지막 인덱스는 계산에서 제외.
      loss_fct = nn.CrossEntropyLoss(ignore_index=-1)

      # logits.view(-1, 2)는 열이 2개 형태로 logits를 정렬
      # next_sentence_label.view(-1)는 행이 하나인 형태로 정렬
      next_sentence_loss = loss_fct(logits.view(-1,2), next_sentence_label.view(-1))
      return next_sentence_loss, logits
    else:
      return logits

In [ ]:
import torch

# colab -> gpu, else -> cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model, config
config = BertConfig()
model = BertEnsembleForNextSentencePrediction(config)

# send model to gpu or cpu
model.to(device)

# load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# set learning rate
learning_rate = 1e-5

# 절편과 가중치를 no_decay변수에 저장
no_decay = ["bias", "LayerNorm.weight"]

# set optimize parameter group
optimizer_grouped_parameters = [{
    "params":[ p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
}]

# optimizer
optimizer = adamw.AdamW(optimizer_grouped_parameters,lr=learning_rate)

In [ ]:
# prepare_data function
def prepare_data(dataset, qa=True):
  # empty list
  input_ids, attention_masks = [], []
  labels = []

  for point in dataset:
    if qa is True:
      # point에 있는 3개의 원소를 앞에서부터 q, a, _로 할당
      q, a, _ = point
    else :
      a, q, _ = point

    # q, a encode by tokenizer
    encoded_dict = tokenizer.encode_plus(
        q, # text1 encode
        a,  # text2 encode
        add_special_tokens = True,  #  speicial token[CLS], [SEP] add
        max_length=128,
        pad_to_max_length=True,   # padding to max_length
        return_attention_mask = True,
        return_tensors = 'pt',  #pt -> pytorch
        truncation=True
    )

    # encoded_dict("input_ids")를 컨테이너 변수 input_ids에 순서대로 저장
    input_ids.append(encoded_dict["input_ids"])

    # encoded_dict("attention_mask")를 attention_masks에 순서대로 저장
    attention_masks.append(encoded_dict["attention_mask"])

    # point의 마직막 (세번째) 원소(레이블)을 labels에 순서대로 저장.
    labels.append(point[-1])

    # end for loop

  # input_ids 첫번째 축(dim=0), 세로방향으로 병합
  input_ids = torch.cat(input_ids, dim=0)

  #attention_mask도 첫번째 축(dim=0), 새로방향으로 병합
  attention_masks = torch.cat(attention_masks, dim=0)

  # return input_ids, attention_masks, labels
  return input_ids, attention_masks, labels

In [ ]:
import numpy as np
from torch.utils.data import DataLoader, RandomSampler, Dataset, SequentialSampler

# QADataset class
class QADataset(Dataset):

  # constructor
  def __init__(self, input_ids, attention_masks, labels=None):
    self.input_ids = np.array(input_ids)
    self.attention_masks = np.array(attention_masks)
    # torch.long은 정수타입을 의미
    self.labels = torch.tensor(labels, dtype=torch.long) if labels is not None else None

  def __get_item__(self, index):
    return self.input_ids[index], self.attention_mask[index], self.labels[index] if self.labels is not None else None

  def __len__(self):
    return self.input_ids.shape[0]